In [9]:
from CONSTANT import *
from tools import *
import pandas as pd
import numpy as np
from torch.utils.data import (TensorDataset, DataLoader, SequentialSampler, WeightedRandomSampler)

import torch

from torch import nn

use_cuda = True
valid='loso'

device = torch.device('cuda' if torch.cuda.is_available() and use_cuda else 'cpu')
print('using', device)

using cuda


In [17]:
class DataPrepare(object):
    def __init__(self, target, data, train_index, test_index, batch_size=64,):

        X, y = join_signals(data, target=target)
        xtrain, ytrain, xtest, ytest = X[train_index], y[train_index], X[test_index], y[test_index]
        print(xtrain.shape, ytrain.shape, xtest.shape, ytest.shape)
        
        self.xtrain = torch.from_numpy(xtrain).to(device).to (torch.float32)
        self.xtest = torch.from_numpy(xtest).to(device).to (torch.float32)
        
        self.ytrain = torch.from_numpy(ytrain).to(device).to (torch.float32)
        self.ytest = torch.from_numpy(ytest).to(device).to (torch.float32)
        
        print(self.xtrain.isnan().any(), self.xtest.isnan().any(), 
              self.ytrain.isnan().any(), self.ytest.isnan().any(),)
        
        self.batch_size = batch_size
        
    
    def get_data(self):
        train_data = TensorDataset(self.xtrain, self.ytrain)
        test_data = TensorDataset(self.xtest, self.ytest)
        
        train_sampler = SequentialSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=self.batch_size, drop_last=True)
        
        test_sampler = SequentialSampler(test_data)
        test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=self.batch_size, drop_last=True)

        return train_dataloader, test_dataloader

In [12]:
spliter = load_model(r'./processed_signal/HKU956/400_4s_step_2s_spliter.pkl')


# data = pd.read_pickle(r'./processed_signal/HKU956/400_4s_step_2s.pkl')
data = pd.read_csv(r'./processed_signal/HKU956/400_4s_step_2s.csv')

for k in spliter[valid]:
    train_index = k['train_index']
    test_index = k['test_index']
    break

In [18]:
dataprepare = DataPrepare(target='valence', data=data, train_index=train_index, test_index=test_index)
train_dataloader, test_dataloader = dataprepare.get_data()

(18089, 4, 400) (18089,) (4638, 4, 400) (4638,)
tensor(False, device='cuda:0') tensor(False, device='cuda:0') tensor(False, device='cuda:0') tensor(False, device='cuda:0')


In [7]:
class Params(object):
    def __init__(self, lr=0.001, epoch=100, valid='loso', target='valence', batch_size=64,):
        self.batch_size = batch_size
        self.valid = valid
        self.target = target
        self.epoch = epoch
        self.lr = lr
        self.metrics_dict = {}

In [31]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
xtest = dataprepare.xtest[:2]
xtest.shape

torch.Size([2, 4, 400])

In [20]:
xtest = xtest.to(device)

In [60]:
# https://blog.csdn.net/Cyril_KI/article/details/125439045
# https://github.com/ozancanozdemir/CNN-LSTM/blob/main/cnn-lstm.py

cnns = nn.Sequential(
    nn.Conv1d(4, 32, 3),
    nn.ReLU(inplace=True),
    nn.MaxPool1d(kernel_size=2, stride=1),
    nn.BatchNorm1d(32),
    nn.Conv1d(32, 64, 3),
    nn.MaxPool1d(kernel_size=2, stride=1),
    nn.BatchNorm1d(64),
    nn.ReLU(inplace=True),
)

lstm1 = nn.LSTM(input_size=64, 
            hidden_size=256,
            num_layers=3, batch_first=True,
            bidirectional=True, dropout=0.5
            )

lstm2 = nn.LSTM(input_size=256, 
            hidden_size=256,
            num_layers=3, batch_first=True,
            bidirectional=True, dropout=0.5
            )
    

classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 2)
        )

cnns.to(device)
lstms.to(device)
classifier.to(device)

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=256, out_features=1024, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=1024, out_features=512, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=512, out_features=2, bias=True)
)

In [78]:
lstm = nn.Sequential(nn.LSTM(input_size=64, 
            hidden_size=256,
            num_layers=3, batch_first=True,
            bidirectional=True, dropout=0.5
            )).to(device)

lstm2 = nn.LSTM(input_size=512, 
            hidden_size=512,
            num_layers=3, batch_first=True,
            bidirectional=True, dropout=0.5
            ).to(device)

bn1 = nn.Sequential(nn.BatchNorm1d(256*2),
                    nn.ReLU(inplace=True),
        ).to(device)

bn2 = nn.Sequential(nn.BatchNorm1d(512*2),
                    nn.ReLU(inplace=True),
        ).to(device)

In [80]:
x = cnns(xtest) # output [batch_size, channels, seq_len]
x = x.permute(0, 2, 1)
x, _ = lstm(x) # output [batch_size, seq_len, Hin]
x = x.permute(0, 2, 1)
x = bn1(x)  # output [batch_size, Hin, seq_len]
x = x.permute(0, 2, 1)
x, _ = lstm2(x) # output [batch_size, seq_len, Hin]
x = x.permute(0, 2, 1)
x = bn2(x) # output [batch_size, Hin, seq_len]

In [82]:
classifier(x).shape

torch.Size([2, 1024, 2])

In [ ]:
x = cnns(xtest) # output [batch_size, channels, seq_len]
x = x.permute(0, 2, 1)
x, _ = lstms(x) # output [batch_size, seq_len, Hin]
# classifier(x).shape

In [57]:
x.shape

torch.Size([2, 394, 512])

In [ ]:
cnns(xtest)

In [92]:
class CNNBiLSTM(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.cnns = nn.Sequential(
            nn.Conv1d(4, 32, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=1),
            nn.BatchNorm1d(32),
            nn.Conv1d(32, args.out_channels, 3),
            nn.MaxPool1d(kernel_size=2, stride=1),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
        )

        self.lstm1 = nn.LSTM(input_size=args.out_channels, 
                    hidden_size=512,
                    num_layers=args.num_layers, batch_first=True,
                    bidirectional=True
                    )

        self.lstm2 = nn.LSTM(input_size=512 * 2, 
                    hidden_size=args.hidden_size,
                    num_layers=args.num_layers, batch_first=True,
                    bidirectional=True
                    )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(args.hidden_size, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 2)
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """
        input [batch_size, channels, seq_len]
        """
        x = self.cnns(x) # output [batch_size, channels, seq_len]
        x = x.permute(0, 2, 1)
        x, _ = self.lstm1(x) # output [batch_size, seq_len, Hin]
        x = self.relu(x)
        x, _ = self.lstm2(x) # output [batch_size, seq_len, Hin]
        x = self.relu(x)
        output = classifier(x)
        return output

In [93]:
class Params(object):
    def __init__(self):
        super().__init__()
        self.out_channels = 64
        self.hidden_size = 256
        self.num_layers = 3

In [94]:
args = Params()
net = CNNBiLSTM(args).to(device)

In [96]:
net(xtest).shape

torch.Size([2, 394, 2])